### Compare loops across celltypes

In [1]:
import pybedtools

In [ ]:
def make_bt(filename, celltype):

    ranges = []
    slop = 2000

    with open(filename, 'r') as f:
        for line in f:
            line = line.strip('\n').split('\t')
            updated_id = celltype + ':' + line[-1] 
            start1 = int(line[1]) - slop
            end1 = int(line[2]) + slop
            start2 = int(line[4]) - slop
            end2 = int(line[5]) + slop
            ranges.append([line[0], start1, end1, line[0], start2, end2, updated_id])

    bt = pybedtools.BedTool(ranges)
    
    return bt

In [3]:
bt1 = make_bt('GM12878_rcmc_all_1kb.bedpe', 'GM12878')
bt2 = make_bt('HCT116_rcmc_all_1kb.bedpe', 'HCT116')
bt3 = make_bt('H1_rcmc_all_1kb.bedpe', 'H1')
bt4 = make_bt('K562_rcmc_all_1kb.bedpe', 'K562')

# bt1 = make_bt('../predicted_merged_loopcalls/GM12878_genome_wide_0.001_pred_merged_loops.txt', 'GM12878')
# bt2 = make_bt('../predicted_merged_loopcalls/HCT116_genome_wide_0.001_pred_merged_loops.txt', 'HCT116')
# bt3 = make_bt('../predicted_merged_loopcalls/H1_genome_wide_0.001_pred_merged_loops.txt', 'H1')
# bt4 = make_bt('../predicted_merged_loopcalls/K562_genome_wide_0.001_pred_merged_loops.txt', 'K562')

In [4]:
def cleaned_intersect(x, y, z, q, min_fraction = 0.5):
    """
    modified from https://daler.github.io/pybedtools/_modules/pybedtools/contrib/venn_maker.html#cleaned_intersect
    """

    # Same as 2-way
    unique_to_y = y.pair_to_pair(x, type='notboth', f=min_fraction)
    y_shared_with_x = x.pair_to_pair(y, type='both', f=min_fraction)
    # postmerge controls whether the intervals are merged after concatenation
    new_y = unique_to_y.cat(y_shared_with_x, postmerge=False)

    # Same as 3-way
    unique_to_z = z.pair_to_pair(x, type='notboth', f=min_fraction).pair_to_pair(y, type='notboth', f=min_fraction)
    z_shared_with_x = x.pair_to_pair(z, type='both', f=min_fraction)
    to_remove1 = z.pair_to_pair(x, type='both', f=min_fraction)
    z_shared_with_unique_y = unique_to_y.pair_to_pair(z, type='both', f=min_fraction)
    z_shared_with_unique_y_distinct = z_shared_with_unique_y.pair_to_pair(to_remove1, type='notboth', f=min_fraction)

    new_z = unique_to_z.cat(z_shared_with_x, postmerge=False).cat(z_shared_with_unique_y_distinct, postmerge=False)
    # Combine:
    #  unique-to-q
    #  shared-with-any-x
    #  shared-with-unique-to-y
    #  shared-with-unique-to-z
    unique_to_q = q.pair_to_pair(z, type='notboth', f=min_fraction).pair_to_pair(y, type='notboth', f=min_fraction).pair_to_pair(x, type='notboth', f=min_fraction)
    q_shared_with_x = x.pair_to_pair(q, type='both', f=min_fraction)
    to_remove2 = q.pair_to_pair(x, type='both', f=min_fraction)
    q_shared_with_unique_y = unique_to_y.pair_to_pair(q, type='both', f=min_fraction)
    q_shared_with_unique_y_distinct = q_shared_with_unique_y.pair_to_pair(to_remove2, type='notboth', f=min_fraction)
    to_remove3 = q.pair_to_pair(q_shared_with_unique_y_distinct, type='both', f=min_fraction)
    q_shared_with_unique_z = unique_to_z.pair_to_pair(q, type='both', f=min_fraction)
    q_shared_with_unique_z_distinct = q_shared_with_unique_z.pair_to_pair(to_remove2, type='notboth', f=min_fraction).pair_to_pair(to_remove3,  type='notboth', f=min_fraction)

    new_q = unique_to_q.cat(q_shared_with_x, postmerge=False).cat(q_shared_with_unique_y_distinct, postmerge=False).cat(q_shared_with_unique_z_distinct, postmerge=False)
    
    return x, new_y, new_z, new_q

In [5]:
updated = cleaned_intersect(bt1, bt2, bt3, bt4, min_fraction=0.3)

In [ ]:
updated[0].saveas('GM12878_rcmc_all_2kb_updated.bedpe')
updated[1].saveas('HCT116_rcmc_all_2kb_updated.bedpe')
updated[2].saveas('H1_rcmc_all_2kb_updated.bedpe')
updated[3].saveas('K562_rcmc_all_2kb_updated.bedpe')

# updated[0].saveas('../predicted_merged_loopcalls/updated_loops_for_comparisons/GM12878_genome_predicted_2kb_updated.bedpe')
# updated[1].saveas('../predicted_merged_loopcalls/updated_loops_for_comparisons/HCT116_genome_predicted_2kb_updated.bedpe')
# updated[2].saveas('../predicted_merged_loopcalls/updated_loops_for_comparisons/H1_genome_predicted_2kb_updated.bedpe')
# updated[3].saveas('../predicted_merged_loopcalls/updated_loops_for_comparisons/K562_genome_predicted_2kb_updated.bedpe')

### Loop pileup analyses

In [ ]:
import cooler
import cooltools
import coolpuppy
from coolpuppy import coolpup
from coolpuppy import plotpup
import pandas as pd
import bioframe
import matplotlib.pyplot as plt
import numpy as np
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

plt.rc('pdf',fonttype = 42)
from matplotlib.font_manager import FontProperties

# change or remove to not use Helvetica
default_font = FontProperties(fname="/mnt/md0/clarice/packages/fonts/Helvetica.ttf")

In [ ]:
celltypes = ['GM12878', 'HCT116', 'K562', 'H1']

In [ ]:
rcmc_clrs = {}

for celltype in celltypes:
    rcmc_clrs[celltype] = cooler.Cooler(f'/mnt/md0/clarice/realigned_rcmc_merged/{celltype}_merged_realigned.50.mcool::resolutions/200')

In [ ]:
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
hg38_cens = bioframe.fetch_centromeres('hg38')
hg38_arms = bioframe.make_chromarms(hg38_chromsizes, hg38_cens)
hg38_arms = hg38_arms[hg38_arms.chrom.isin(rcmc_clrs['GM12878'].chromnames)].reset_index(drop=True)
hg38_arms = hg38_arms[hg38_arms['chrom'] != 'chrY']

In [ ]:
region_df = pd.read_csv('/mnt/md0/clarice/src/region_idx.txt', sep='\t').rename(columns={'region_id':'name'})
region_df = bioframe.sort_bedframe(region_df, view_df=hg38_chromsizes)

In [ ]:
all_loops = pd.read_csv('../example_data/loop_examples/all_rcmc_annotated_loops.tsv', sep = '\t')
all_loops.rename(columns={'seqnames1': 'chrom1', 'seqnames2': 'chrom2'})

all_loops_by_celltype = {}

for celltype in celltypes:
    all_loops_by_celltype[celltype] = all_loops[all_loops['celltype'] == celltype]

In [ ]:
expected_dfs = {}

for celltype in celltypes:
    expected_dfs[celltype] = cooltools.expected_cis(
                        clr=rcmc_clrs[celltype],
                        view_df=region_df,
                        smooth=True,
                        aggregate_smoothed=True,
                        smooth_sigma=0.1,
                        nproc=16
                    )


In [ ]:
# all loop pileups
all_pups = {}
full_pups = {}

for celltype, clr in rcmc_clrs.items():
    for loop_celltype, loops in all_loops_by_celltype.items():
        name = f'{celltype}_{loop_celltype}'
        pup = coolpup.pileup(clr, loops, features_format='bedpe', view_df=region_df, expected_df=expected_dfs[celltype], nproc=8, flank=20000)
        full_pups[name] = pup
        all_pups[name] = pup['data'][0]

In [ ]:
# E-P loop pileups
ep_pups = {}

ep_loops = {}
for celltype, loops in all_loops_by_celltype.items():
    ep_loops[celltype] = loops.query("loop_class == ['E-P']")

for celltype, clr in rcmc_clrs.items():
    for loop_celltype, loops in ep_loops.items():
        name = f'{celltype}_{loop_celltype}'
        pup = coolpup.pileup(clr, loops, features_format='bedpe', view_df=region_df, expected_df=expected_dfs[celltype], nproc=8, flank=20000)
        ep_pups[name] = pup['data'][0]

In [ ]:
# CTCF loop pileups
ctcf_pups = {}

ctcf_loops = {}
for celltype, loops in all_loops_by_celltype.items():
    ctcf_loops[celltype] = loops.query("loop_class == ['CTCF-CTCF']")

for celltype, clr in rcmc_clrs.items():
    for loop_celltype, loops in ctcf_loops.items():
        name = f'{celltype}_{loop_celltype}'
        pup = coolpup.pileup(clr, loops, features_format='bedpe', view_df=region_df, expected_df=expected_dfs[celltype], nproc=8, flank=20000)
        ctcf_pups[name] = pup['data'][0]

In [ ]:
def get_enrichment(amap, n, dec=2):
    """
    Function directly from the coolpuppy package
    """
    c = int(np.floor(amap.shape[0]/2))
    return np.round(np.nanmean(amap[c-n//2:c+n//2+1, c-n//2:c+n//2+1]), decimals=dec)

def annotate_enrichment(ax, amap, n, dec=2, size=6, bold=False):
    """
    Function modified from the coolpuppy package, to control where the enrichment annotation goes
    """
    enr = get_enrichment(amap, n, dec)
    if bold == False:
        ax.text(10, 40, enr, ha='left', va='bottom', fontsize=size, fontproperties = default_font)
    else:
        ax.text(10, 40, enr, ha='left', va='bottom', fontsize=size, fontproperties = FontProperties(fname="/mnt/md0/clarice/packages/fonts/helvetica-bold.ttf"))

In [ ]:
# Plot pileup
fig, axs = plt.subplots(figsize=[10, 5],
                        nrows=4,
                        ncols=4)

# limits empirically determined - change for all/EP/CTCF plot

# limits for all loops
loop_limits = {'GM12878': (-0.5, 1.5), 'HCT116': (-0.2, 1.6), 'K562': (-0.5, 1.4), 'H1': (-0.8, 2)}
# limits for E-P loops
# loop_limits = {'GM12878': (-0.5, 1.5), 'HCT116': (-0.2, 1.4), 'K562': (-0.8, 1.2), 'H1': (-0.3, 1.8)}
# limits for CTCF loops
# loop_limits = {'GM12878': (-1, 2.7), 'HCT116': (-1, 2.8), 'K562': (-1, 3), 'H1': (-1, 3)}

for row_idx, loop_celltype in enumerate(celltypes):
    for col_idx, celltype in enumerate(celltypes):
        name = f'{celltype}_{loop_celltype}'
        ax = axs[row_idx, col_idx]
        amin, amax = loop_limits[loop_celltype]
        # change for all/EP/CTCF loops
        data = all_pups[name]
        m = ax.imshow(np.log2(data), cmap='coolwarm', vmax = amax, vmin = amin)
        ax.set_yticks([], [])
        ax.set_xticks([], [])

        if celltype == loop_celltype:
            annotate_enrichment(ax, data, 3, size=10, bold=True)
        else:
            annotate_enrichment(ax, data, 3, size=10)

        if row_idx == 0:
            ax.set_title(celltype, rotation=0, size=12, fontproperties = default_font)
        if col_idx == 0:
            ax.set_ylabel(loop_celltype, size=12, fontproperties = default_font)
        
    # Define a new axis for the colorbar using `add_axes`
    cax = fig.add_axes([ax.get_position().x1 - 0.15,  # x-position of colorbar
                        ax.get_position().y0,        # y-position of colorbar
                        0.01,                        # width of colorbar
                        ax.get_position().height])   # height of colorbar

    cb = plt.colorbar(m, cax=cax)
    mid = round((amin + amax)/2, 2)
    cb.set_ticks([amin, mid, amax])
    cb.set_ticklabels([amin, mid, amax], fontproperties = default_font)
    cb.ax.minorticks_off()

# change for all/EP/CTCF loops
plt.text(-43, 3.5, 'loops called in', fontsize=14, rotation = 'vertical', fontproperties = default_font)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=-0.8, hspace=None)
plt.savefig('figures/all_loop_pileup.pdf')